In [268]:
#!/usr/bin/env python3
import argparse
import asyncio
import os
import random
import sys
from sys import platform

import keras
import music21
import numpy as np
import pygame
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Dense, Embedding, TimeDistributed, Dropout, Flatten, Input
from keras.models import Sequential, Model
from loguru import logger
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from midi2audio import FluidSynth
from keras.layers import concatenate
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
import pandas



In [269]:
def setup_logging(level="DEBUG", show_module=False):
    """
    Setups better log format for loguru
    """
    logger.remove(0)    # Remove the default logger
    log_level = level
    log_fmt = u"<green>["
    log_fmt += u"{file:10.10}…:{line:<3} | " if show_module else ""
    log_fmt += u"{time:HH:mm:ss.SSS}]</green> <level>{level: <8}</level> | <level>{message}</level>"
    logger.add(sys.stderr, level=log_level, format=log_fmt, colorize=True, backtrace=True, diagnose=True)



def play_midi(midi_file):

    #Play MIDI
    sf2_path = '/usr/share/soundfonts/freepats-general-midi.sf2'  # path to sound font file
    FluidSynth(sound_font=sf2_path).play_midi(midi_file)


In [270]:
sequence_length = 16
batch_size = 16
epochs = 300
# Load the musical data using Music21
corpi = [c for c in music21.corpus.getComposer("mozart")[1:5]]

X_pitch = []
X_duration = []
y_pitch = []
y_duration = []
notes = []


ql_to_ordinal = {}
ordinal_to_ql = {}
# for piece in corpus:
for corpus in corpi:
    logger.info(f'Parsing {corpus}')
    parsed = music21.converter.parse(corpus)
    elements = parsed.flat.notes
    for element in tqdm(elements):
        if isinstance(element, music21.note.Note):
            # note = (element.pitch.midi, element.duration.quarterLength)
            # notes.append(note)
            ql_to_ordinal[element.duration.quarterLength] = element.duration.ordinal
            ordinal_to_ql[element.duration.ordinal] = element.duration.quarterLength
            notes.append((element.pitch.pitchClass, element.pitch.octave, element.duration.ordinal))

# stream = music21.stream.Stream()
# for _note in notes:
#     pitch = music21.pitch.Pitch(_note[0], octave=_note[1])
#     duration = music21.duration.Duration(_note[2])
#     _note_object = music21.note.Note(pitch=pitch, duration=duration)
#     stream.append(_note_object)
# stream.write('midi', fp='generated_music.mid')
# play_midi('generated_music.mid')

# Preprocess the data
X_pitch = []
X_octave = []
X_duration = []
y_pitch = []
y_octave = []
y_duration = []
for i in tqdm(range(len(notes) - sequence_length)):
    X_pitch.append([pitch for pitch, octave, duration in notes[i:i + sequence_length]])    
    X_octave.append([octave for pitch, octave, duration in notes[i:i + sequence_length]])    
    X_duration.append([duration for pitch, octave, duration in notes[i:i + sequence_length]])
    y_pitch.append(notes[i + sequence_length][0])
    y_octave.append(notes[i + sequence_length][1])
    y_duration.append(notes[i + sequence_length][2])


pitch_classes_count = max(np.max(X_pitch), np.max(y_pitch)) + 1
octave_classes_count = max(np.max(X_octave), np.max(y_octave)) + 1
duration_classes_count = max(np.max(X_duration), np.max(y_duration)) + 1

X_pitch = to_categorical(X_pitch, num_classes=pitch_classes_count)
X_octave = to_categorical(X_octave, num_classes=octave_classes_count)
X_duration = to_categorical(X_duration, num_classes=duration_classes_count)

y_pitch = to_categorical(y_pitch, num_classes=pitch_classes_count)
y_octave = to_categorical(y_octave, num_classes=octave_classes_count)
y_duration = to_categorical(y_duration, num_classes=duration_classes_count)

# Convert the data to numpy arrays
X_pitch = np.array(X_pitch)
X_octave = np.array(X_octave)
X_duration = np.array(X_duration)
y_pitch = np.array(y_pitch)
y_octave = np.array(y_octave)
y_duration = np.array(y_duration)

n_samples = (len(X_pitch) // batch_size) * batch_size
X_pitch = X_pitch[:n_samples]
X_octave = X_octave[:n_samples]
X_duration = X_duration[:n_samples]
y_pitch = y_pitch[:n_samples]
y_octave = y_octave[:n_samples]
y_duration = y_duration[:n_samples]


2023-02-25 19:14:48.075 | INFO     | __main__:<module>:18 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/mozart/k155/movement2.mxl
100%|██████████| 930/930 [00:00<00:00, 22020.95it/s]
2023-02-25 19:14:48.202 | INFO     | __main__:<module>:18 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/mozart/k155/movement3.mxl
100%|██████████| 1121/1121 [00:00<00:00, 21759.90it/s]
2023-02-25 19:14:48.372 | INFO     | __main__:<module>:18 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/mozart/k156/movement1.mxl
100%|██████████| 1371/1371 [00:00<00:00, 20704.45it/s]
2023-02-25 19:14:49.262 | INFO     | __main__:<module>:18 - Parsing /home/emi/Coding/melodemi/.venv/lib/python3.10/site-packages/music21/corpus/mozart/k156/movement2.mxl
100%|██████████| 4435/4435 [00:00<00:00, 300464.19it/s]


In [271]:
# Define the model architecture
input_pitch = Input(shape=(sequence_length, pitch_classes_count))
input_duration = Input(shape=(sequence_length, duration_classes_count))
input_octave = Input(shape=(sequence_length, octave_classes_count))
merged = concatenate([input_pitch, input_octave, input_duration])

lstm1 = LSTM(units=128, activation='tanh', return_sequences=True)(merged)
lstm2 = LSTM(units=128)(lstm1)
output_pitch = Dense(pitch_classes_count, activation='softmax', name='pitch')(lstm2)
output_octave = Dense(octave_classes_count, activation='softmax', name='octave')(lstm2)
output_duration = Dense(duration_classes_count, activation='softmax', name='duration')(lstm2)

model = Model(inputs=[input_pitch, input_octave, input_duration], outputs=[output_pitch, output_octave, output_duration])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Train the model
history = model.fit([X_pitch, X_octave, X_duration], [y_pitch, y_octave, y_duration], batch_size=batch_size, epochs=epochs)


Epoch 1/300
277/277 [==============================] - 5s 11ms/step - loss: 4.5737 - pitch_loss: 2.2346 - octave_loss: 1.2401 - duration_loss: 1.0990 - pitch_categorical_accuracy: 0.1717 - octave_categorical_accuracy: 0.4375 - duration_categorical_accuracy: 0.5871
Epoch 2/300
277/277 [==============================] - 3s 11ms/step - loss: 4.3109 - pitch_loss: 2.1559 - octave_loss: 1.1790 - duration_loss: 0.9760 - pitch_categorical_accuracy: 0.1832 - octave_categorical_accuracy: 0.4725 - duration_categorical_accuracy: 0.6207
Epoch 3/300
277/277 [==============================] - 3s 11ms/step - loss: 4.1968 - pitch_loss: 2.1145 - octave_loss: 1.1519 - duration_loss: 0.9304 - pitch_categorical_accuracy: 0.2074 - octave_categorical_accuracy: 0.4822 - duration_categorical_accuracy: 0.6381
Epoch 4/300
277/277 [==============================] - 3s 11ms/step - loss: 4.0909 - pitch_loss: 2.0749 - octave_loss: 1.1252 - duration_loss: 0.8908 - pitch_categorical_accuracy: 0.2137 - octave_categoric

In [294]:
import numpy as np

def predict_sequence_with_duration(model, seed_sequence, sequence_length):
    # Encode the seed sequence
    seed_sequence_pitch = [pitch for pitch, octave, duration in seed_sequence]
    seed_sequence_octave = [octave for pitch, octave, duration in seed_sequence]
    seed_sequence_duration = [duration for pitch, octave, duration in seed_sequence]
    seed_sequence_pitch = to_categorical([seed_sequence_pitch], num_classes=pitch_classes_count)
    seed_sequence_octave = to_categorical([seed_sequence_octave], num_classes=octave_classes_count)
    seed_sequence_duration = to_categorical([seed_sequence_duration], num_classes=duration_classes_count)

    # Reshape the seed sequence for the model
    # seed_sequence_pitch = seed_sequence_pitch.reshape(1, sequence_length, pitch_classes_count)
    # seed_sequence_octave = seed_sequence_octave.reshape(1, sequence_length, duration_classes_count)
    # seed_sequence_duration = seed_sequence_duration.reshape(1, sequence_length, duration_classes_count)

    # Predict the next pitch and duration in the sequence
    predicted_pitch, predicted_octave, predicted_duration = model.predict([seed_sequence_pitch, seed_sequence_octave, seed_sequence_duration], verbose=0)

    # Decode the predicted pitch and duration
    next_pitch = np.argmax(predicted_pitch[0])
    next_octave = np.argmax(predicted_octave[0])
    next_duration = np.argmax(predicted_duration[0])

    return next_pitch, next_octave, next_duration

def generate_song_with_duration(model, seed_sequence, sequence_length, song_length):
    song = seed_sequence
    for i in range(song_length - sequence_length):
        next_pitch, next_octave, next_duration = predict_sequence_with_duration(model, song[-sequence_length:], sequence_length)
        print(next_pitch, next_octave, next_duration)
        song.append((next_pitch, next_octave, next_duration))
    return song

# Set the seed sequence and the desired length of the generated song
# seed_sequence = [(60, 1), (62, 1), (64, 1), (65, 1), (67, 0.5), (69, 0.5), (71, 0.5), (72, 0.5)] * 4

seed_sequence = []
for i in reversed(range(sequence_length)):
    pitch = notes[i][0]
    octave = notes[i][1]
    duration = notes[i][2]
    seed_sequence.append((pitch, octave, duration))
song_length = 256

# Generate the song
song = generate_song_with_duration(model, seed_sequence, sequence_length, song_length)


9 3 7
11 2 7
6 5 7
9 3 7
6 4 8
2 5 8
2 4 8
11 3 8
1 5 8
9 3 8
1 5 8
9 4 8
9 2 8
2 5 8
6 3 8
4 5 8
8 4 8
4 5 7
1 4 7
9 3 7
11 2 6
2 4 7
11 4 7
9 2 7
11 3 7
1 4 7
9 2 7
4 4 8
1 4 8
9 3 7
4 4 6
2 3 7
4 3 6
1 4 7
9 3 7
9 4 7
6 4 7
11 3 7
9 4 7
6 4 7
9 3 7
9 3 6
2 4 7
11 3 7
2 4 7
4 4 7
4 4 7
9 3 7
1 4 6
9 4 7
2 3 6
4 4 7
1 4 7
4 3 7
9 3 6
6 4 7
2 4 7
2 3 7
9 4 6
9 3 7
2 3 7
8 4 8
4 5 7
9 4 7
6 3 7
2 4 7
11 3 7
2 4 7
11 3 7
2 2 7
9 4 7
6 4 7
1 4 7
9 2 6
9 3 6
2 4 7
2 4 7
9 4 5
4 4 7
4 4 7
9 2 6
9 3 6
6 4 7
6 4 7
7 4 7
1 4 7
9 3 6
7 3 6
4 4 7
7 4 6
2 4 6
6 3 6
2 3 6
9 4 7
2 4 7
9 5 7
6 4 7
7 4 7
4 4 7
2 4 7
9 5 7
6 4 7
9 5 7
9 4 7
6 4 7
2 4 7
7 5 7
9 4 7
6 3 6
2 5 6
7 5 6
11 4 7
7 4 7
2 4 7
11 4 7
2 4 7
11 5 7
11 4 7
7 4 6
2 5 6
11 5 7
11 4 7
7 5 7
2 4 7
11 4 7
7 4 7
0 5 7
9 4 7
2 4 7
6 5 6
9 3 6
0 5 7
2 4 7
0 5 7
9 4 7
0 5 7
2 4 7
9 3 6
11 3 6
0 5 7
7 4 7
9 4 7
6 4 7
9 5 6
2 4 6
2 3 6
9 4 7
6 4 7
2 4 7
6 4 7
2 5 7
9 4 6
6 4 6
2 4 6
6 4 7
2 4 7
9 4 7
6 4 7
6 5 7
9 4 7
6 4 7
2 4 7
2 5 7
9 4 7

In [296]:
# Write the generated stream to a MIDI file
stream = music21.stream.Stream()
for _note in song:
    pitch = music21.pitch.Pitch(_note[0], octave=_note[1])
    _note_object = music21.note.Note(pitch=pitch, duration=music21.duration.Duration(ordinal_to_ql[_note[2]]))
    stream.append(_note_object)

stream.write('midi', fp='generated_music.mid')

play_midi('generated_music.mid')

ALSA lib pcm_dsnoop.c:566:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:999:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm_

KeyboardInterrupt: 